In [1]:
import torch
import pandas as pd
import pyaging as pya
import os

In [2]:
os.system("git clone https://github.com/MorganLevineLab/methylCIPHER.git")

0

In [7]:
df = pd.read_csv('methylCIPHER/data-raw/HRSInChPhenoAge_CpG.csv')

df['feature'] = df['CpG']
df['coefficient'] = df['Weight']

In [8]:
features = df['feature'].tolist()

weights = torch.tensor(df['coefficient'].tolist()).unsqueeze(0)
intercept = torch.tensor([52.8334080])

In [9]:
model = pya.models.LinearModel(len(features))

model.linear.weight.data = weights
model.linear.bias.data = intercept

model

LinearModel(
  (linear): Linear(in_features=959, out_features=1, bias=True)
)

In [10]:
weights_dict = {
    'preprocessing': None, 
    'preprocessing_helper': None,
    'postprocessing': None,
    'postprocessing_helper': None,
    'features': features,
    'weight_dict': model.state_dict(),
    'model_class': 'LinearModel',
}

metadata_dict = {
    'species': 'Homo sapiens',
    'data_type': 'methylation',
    'year': 2022,
    'implementation_approved_by_author(s)': '⌛',
    'preprocessing': weights_dict['preprocessing'], 
    'postprocessing': weights_dict['postprocessing'], 
    'citation': "Higgins-Chen, Albert T., et al. \"A computational solution for bolstering reliability of epigenetic clocks: Implications for clinical trials and longitudinal tracking.\" Nature aging 2.7 (2022): 644-661.",
    'doi': "https://doi.org/10.1038/s43587-022-00248-2",
    "notes": None,
}

In [11]:
torch.save(weights_dict, '../weights/hrsinchphenoage.pt')
torch.save(metadata_dict, '../metadata/hrsinchphenoage.pt')

In [12]:
os.system("rm -r methylCIPHER")

0